<a href="https://colab.research.google.com/github/choprahetarth/background-remover/blob/main/Flask_BG_Remove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook has been created for removing backgrounds using a Flask Web App, where this notebook acts as a back-end.

METHOD - Detectron2 (Facebook)
MODEL USED - 

## Instructions for using `Flask Deployment` based solution - 

1. Switch to a GPU Runtime offered by Google Colab (Runtime-> Change Runtime Type -> GPU )
2. Install the libraries by running the `Install (Run Once) (USE GPU RUNTIME)` Tab
3. Restart the runtime **manually** (it is required, yes.)
4. Expand the `Import Libraries and Upload Images` Tab
5. Run the code till the end. 
6. In the second last cell, you can see something like this 
`` * Running on http://2b55-35-223-110-102.ngrok.io``, the URL can be different slightly, but open it as this is where the Web App has been deployed.
7. In order to do more iterations of the images, you have to close the web-app, and stop the running cell, and run the last cell, then proceed to running the webapp again.

TODO
- Choice for the user to provide color/background (FIX ERRORS)
- Clear the output image.







# Install (Run Once) (USE GPU RUNTIME)

In [1]:
!pip install pyyaml==5.1 > /dev/null
!pip install flask-ngrok > /dev/null
# This is the current pytorch version on Colab. 
# Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html > /dev/null
# exit(0)  # After installation, 
#you need to "restart runtime" in Colab. 
#This line can also restart runtime

# Flask Deployment

### Import Libraries


In [1]:
# Import all the libraries
import numpy as np
import cv2
import requests
import os
import imutils
from PIL import Image
from google.colab.patches import cv2_imshow

In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import torch, torchvision
import json, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### Functions 

In [36]:
# Streamlined Code
def read_image(image,bg,size):
  bg = bg
  image = image
  foreground = cv2.imread(image)
  background = cv2.imread(bg)
  foreground = imutils.resize(foreground, width=size)
  background = imutils.resize(background, width=size)

  return foreground,background

In [37]:
def config(image):
  image = image
  cfg = get_cfg()
  # add project-specific config (e.g., TensorMask) 
  # here if you're not running a model in detectron2's core library
  cfg.merge_from_file(model_zoo.get_config_file(YAML))
  # set threshold for this model
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  
  # Find a model from detectron2's model zoo. 
  # You can use the https://dl.fbaipublicfiles... url as well
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(YAML)
  predictor = DefaultPredictor(cfg)
  outputs = predictor(image)
  return outputs,cfg

In [38]:
def mask_blend(outputs):
  outputs = outputs
  mask = outputs["instances"].pred_masks
  # Mask Blender
  total_objects = mask.shape[0]
  # Considering logic that if objects > 1, take top two 
  masks = []
  if (total_objects > 1):
    for i in [0,1]:
      output_mat = outputs['instances'].pred_masks[i].cpu().detach().numpy()
      output_mat = output_mat.astype(np.uint8)
      output_mat = output_mat*255
      masks.append(output_mat)
    mask = cv2.bitwise_or(masks[0],masks[1])
  else:
    output_mat = outputs['instances'].pred_masks[0].cpu().detach().numpy()
    output_mat = output_mat.astype(np.uint8)
    output_mat = output_mat*255
    mask = output_mat
  return mask


In [39]:
def background_replace(image,mask,background_image):
  background_image = background_image
  image = image
  mask = mask
  masked = cv2.bitwise_and(image,image,mask = mask)
  replaced = np.copy(masked)
  if (BACKGROUND == "Color"):
    replaced[mask==0] = BG_COLOR
  elif (BACKGROUND == "Background_Image"):
    masked = Image.fromarray(cv2.cvtColor(masked,
                                          cv2.COLOR_BGR2RGB))
    #masked = Image.fromarray(masked)
    masked = masked.convert("RGBA")
    pixdata = masked.load()
    width, height = masked.size
    for y in range(height):
      for x in range(width):
        if pixdata[x, y] == (0, 0, 0, 255):
            pixdata[x, y] = (255, 255, 255, 0)
    background_image = cv2.resize(background_image,
                                  (image.shape[1],
                                   image.shape[0]))
    background_image = Image.fromarray(cv2.cvtColor(background_image,
                                                    cv2.COLOR_BGR2RGB))
    background_image.paste(masked, (0, 0), masked)
    replaced = background_image
  return replaced

### Deployment

In [8]:
os.chdir('/content')
!mkdir templates
!mkdir -p static/images
os.chdir('/content/templates')
# get the HTML file used
!curl -o index.html https://raw.githubusercontent.com/choprahetarth/background-remover/main/index/index.html
os.chdir('/content')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   565  100   565    0     0   3692      0 --:--:-- --:--:-- --:--:--  3692


In [56]:
from flask import Flask, render_template, request, send_from_directory, send_file
from flask_ngrok import run_with_ngrok
from PIL import Image
import requests
import os
import urllib.request

BG_COLOR = (255,255,255)
BACKGROUND = "Background_Image"
RESIZE = 800
YAML = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"

app = Flask(__name__)
run_with_ngrok(app)   
UPLOAD_PATH = "/content/static/images"
app.config['UPLOAD_FOLDER'] = UPLOAD_PATH

def runner(image,bg_image):
  foreground = image
  background = bg_image
  image,bg_image = read_image(foreground,background,RESIZE)
  outputs,cfg = config(image)
  mask = mask_blend(outputs)
  final = background_replace(image,mask,bg_image)
  return final,bg_image,mask,image 
  
@app.route("/",methods =['GET','POST'])
def upload_predict():
  if request.method == "GET":
      return render_template('index.html')
  if request.method == "POST":
    foreground_image = request.files.get('foreground')
    background_image = request.files.get('background')
    input_data = request.form['color']

    if foreground_image:
      image_location_foreground = os.path.join(UPLOAD_PATH,
                                               "foreground.jpg")
      foreground_image.save(image_location_foreground)
      return render_template('index.html',stage = "Foreground Uploaded")

    if background_image:
      image_location_background = os.path.join(UPLOAD_PATH,
                                               "background.jpg")
      background_image.save(image_location_background)
      return render_template('index.html',stage = "Background Uploaded")
    if input_data:
      BACKGROUND = "Color"
      BG_COLOR = input_data
    if "open" in request.form:
      foreground = os.path.join(UPLOAD_PATH,"foreground.jpg")
      background = os.path.join(UPLOAD_PATH,"background.jpg")
      final,bg_image,mask,image = runner(foreground,background)
      final.save(UPLOAD_PATH+"/output.jpg")
      output_filename = os.path.join(app.config['UPLOAD_FOLDER'],
                                     'output.jpg')
      print(output_filename)
      return render_template('index.html',
                             user_image=output_filename,
                             stage="Done!" )
    
    if "clear" in request.form:
      if ("output.jpg" in os.listdir("/content/static/images/")):
        os.remove("/content/static/images/output.jpg")
      return render_template('index.html', stage="Cleared!" )

  return render_template('index.html',stage = "None")
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5a64-35-223-110-102.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Sep/2021 09:05:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 09:05:55] "GET /static/images/output.jpg HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2021 09:05:55] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2021 09:05:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 09:05:58] "GET /static/images/output.jpg HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2021 09:06:00] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 09:06:00] "GET /static/images/output.jpg HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2021 09:06:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 09:06:06] "GET /static/images/output.jpg HTTP/1.1" 404 -
127.0.0.1 - - [18/Sep/2021 09:06:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 09:06:08] "GET /static/images/output.jpg HTTP/1.1" 404 -
The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}
[2021-09-18 09:06:12,337] ERROR in app: Exception on / [POST]
Trace

In [57]:
os.chdir("/content/static/images")
for f in os.listdir():
  os.remove(f)
os.chdir('/content')